# 네이버 뉴스 크롤링 2018년~2019년까지 웹크롤링및 전처리

In [ ]:
# 간략화

#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
import time

# 크롤링할 url 생성하는 함수 만들기(검색어, 시작 날짜, 종료 날짜, 최대 페이지)
def makeUrl(search, s_date, e_date, maxpage):
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    url = [f"https://search.naver.com/search.naver?where=news&query={search}&sort=0&ds={s_date}&de={e_date}&nso=so%3Ar%2Cp%3Afrom{s_from}to{e_to}%2Ca%3A&start={i}" for i in range(1, int(maxpage) * 10, 10)]
    return url  

# html에서 원하는 속성 추출하는 함수 만들기(기사, 추출하려는 속성값)
def news_attrs_crawler(articles, attrs):
    attrs_content = [i.attrs[attrs] for i in articles]
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html 생성해서 기사 크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스 크롤링 시작#####
search = input("검색어 입력: ")  
s_date = input("시작 날짜 입력(예시: 2019.01.04): ")  #2019.01.04
e_date = input("종료 날짜 입력(예시: 2019.01.05): ")   #2019.01.05
maxpage = input("최대 크롤링할 페이지 수를 입력하세요: ")

# naver url 생성
url = makeUrl(search, s_date, e_date, maxpage)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
news_names = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass


# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    time.sleep(2)

    # 언론사명 가져 오기
    names = news_html.select('#contents > div.copyright > div > p')
    if names:
      name = names[0].string[12:-38]
    elif names != names:
      time.sleep(2)
      names = news_html.select('#content > div.end_ct > div > div.copyright > div > p')
      name = names[0].string[12:-38]
    else:
      name = ''

    # 뉴스 제목 가져 오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져 오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져 오기:  list합치기
    content = ''.join(str(content))

    # html 태그 제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>' # 태그 제거
    pattern2 = r'[\n\t<>]|&lt;|&gt;' # 특수 기호 제거

    title = re.sub(pattern=pattern1, repl='', string=str(title))
    title = re.sub(pattern=pattern2, repl='', string=str(title))

    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern3 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern3, '')
    content = re.sub(pattern2, '', content)[1:-1] # 기사 본문 맨 앞, 맨 뒤 [ ] 제거

    news_names.append(name)
    news_titles.append(title)
    news_contents.append(content)

# 날짜 가져 오기
    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time'][:10]
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='', string=str(news_date))

    news_dates.append(news_date)


###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates, 'name': news_names, 'title':news_titles, 'content':news_contents, 'link': final_urls})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first', ignore_index=True)

#데이터 프레임을 엑셀 파일로 저장
outputFileName = f'{s_date} ~ {e_date}.xlsx'
news_df.to_excel(outputFileName, sheet_name='sheet1')

In [ ]:
import pandas as pd


df1 = pd.read_excel('/content/drive/MyDrive/1차프로젝트 데이터 자료/2018.01.01 ~ 2018.03.31.xlsx')
df2 = pd.read_excel('/content/drive/MyDrive/1차프로젝트 데이터 자료/2018.04.01 ~ 2018.06.30.xlsx')
df3 = pd.read_excel('/content/drive/MyDrive/1차프로젝트 데이터 자료/2018.07.01 ~ 2018.09.30.xlsx')
df4 = pd.read_excel('/content/drive/MyDrive/1차프로젝트 데이터 자료/2018.10.01 ~ 2018.12.31.xlsx')
df5 = pd.read_excel('/content/drive/MyDrive/1차프로젝트 데이터 자료/2019.01.01 ~ 2019.03.31.xlsx')
df6 = pd.read_excel('/content/drive/MyDrive/1차프로젝트 데이터 자료/2019.04.01 ~ 2019.06.30.xlsx')
df7 = pd.read_excel('/content/drive/MyDrive/1차프로젝트 데이터 자료/2019.07.01 ~ 2019.09.30.xlsx')
df8 = pd.read_excel('/content/drive/MyDrive/1차프로젝트 데이터 자료/2019.10.02 ~ 2019.12.31.xlsx')

merged_df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])


merged_df.to_excel('/content/drive/MyDrive/merged_data.xlsx', index=False) 

In [ ]:
# 시계열 데이터로 바꾼후 년도별로 데이터값바꾸기
try:
    df['date'] = pd.to_datetime(df['date'], format='[%Y.%m.%d. %p %I:%M]')
except ValueError:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['date'] = df['date'].fillna(method='ffill')

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')

In [ ]:
df.drop(columns=['link','Unnamed: 0'],inplace=True) # 필요없는 컬럼 없애기

df = df.sort_values(by="date")# date컬럼 정렬

# 본문(content)기준 단어 전처리 


In [ ]:
import pandas as pd
import re
from collections import Counter
from konlpy.tag import Okt

okt = Okt()

# 특수 기호와 숫자를 제거하는 함수
def remove_special_char(text):
    return re.sub('[^ㄱ-ㅣ가-힣a-zA-Z]', '', text)

# 데이터프레임에서 텍스트 열을 형태소 분석하여 딕셔너리로 저장하는 함수
def count_words(text):
    text = remove_special_char(text) # 특수 기호와 숫자 제  거
    morphs = okt.morphs(text)
    stopwords = set(['은', '는', '이', '가', '하다', '되다', '있다', '그', '저', '것', '이러한',
                 '말했다', '부동산','것으로','있다.','수','등','이후','아파트','최근','및','지난','따른'
                 ,'때문에','무단전재','이에','정책','만큼','가능성이','것을','따르면','시장에',
                 '있습니다.','.','현재','그는','것"이라고','대비','기록했다','가격','경우','있는','재배포',
                 '대한','따라','위해','것이다.','있다"고','때문이다.','이는','모두','더',
                 '한', '가격이', '큰', '-', 'ⓒ', '말했다.', '또', '주택', '가장',
                 '미국', '크게', '것은', '특히', '같은', '지난해', '비해',
                 '그러나', '대해', '이상','것이', '높은', '있다는', '올해', 
                 '정부의', '있어', '금지', '밝혔다.', '올', '통해', '시장', 
                 '할', '기자', '등을', '등으로', '가운데', '무단', '중',
                 '일부', '될', '등이', '등의', '때', '위한', '오른', '있다고', '많이', '한다.',
                 '하지만', '중심으로', '게', '향후',
                 '=', '영향을',
                 '각각', '하는', '것이라는', '보이고', '역시', 
                 '반면', '이어', '지역의', '했다.', '다른', '있지만', '않을', '각종', '이번',
                 '많은', '다시', '함께', '없다.', '평균', '이미', '관련',
             'the', '이날', '동안', '볼',
             '더욱', '데', '대해서는', '없는', '않고', 
             '정도', '있기', '나타났다.', '여전히', 
             '전재','전', '후', '주요', '있을',
             '전체', '것이라고', '기존', 
              '아니라', '이를', '있고', '것도', '것”이라고', 
              '불구하고', '설명했다.', '들어', '디지털뉴스이용규칙에', 
              '저작권을', '보이는',
              '된다.', '등에', '보인다.', '기록했다.', '이런',
              '낮은', '보였다.', ':', '인한', '있다”고', '않은', '인해',
              '이하', '계속', '추가', '다소', '지역은', '하고',
              '최대', '필요가', '다만', '우려가', '말','보고',  '내', '보면', '보는',
               '기준으로', '그동안', '등은', '모든', '부담을', 'Copyrights',
                '점을', '한다는', 'Copyright', '/', '총', '&amp;', '거의',
                '또한', '상대적으로', '전재-재배포', '나오고', '있다"며', 'Copyrights', '인근',
                '있으며', '어느', '물론','밝혔다','한다고','것이다','때문이다','머니투데이','오는',
                '집을'])# 불용어 리스트
    morphs = [word for word in morphs if word not in stopwords] # 불용어 제거
    count_dict = Counter(morphs) # Counter 클래스 사용
    return count_dict


# 형태소 분석 결과를 저장할 리스트
morphs_list = []

# 데이터프레임의 각 행을 반복하며 형태소 분석을 수행
for index, row in df.iterrows():
    text = row['content'] # 텍스트 데이터가 저장된 열의 이름에 맞게 수정하세요 # titie이나 conteant로 바꾸기
    morphs_dict = count_words(text)
    morphs_list.append(morphs_dict)

# 모든 단어의 빈도수를 합산한 딕셔너리 생성
total_count = Counter()
for morphs_dict in morphs_list:
    total_count.update(morphs_dict)

# 결과 출력하기
for word, count in sorted(total_count.items(), key=lambda x: x[3], reverse=True):
    if len(word) >= 2 and count >= 100: # word = 글자수  count=글자빈도수 설정
        print(word, count)

# morphs_list를 데이터프레임으로 변환
df_morphs = pd.DataFrame(morphs_list)

# 각 단어의 총 빈도수 계산
total_count = df_morphs.sum(axis=0)

# 총 빈도수를 데이터프레임으로 변환
df_total_count = pd.DataFrame({'word': total_count.index, 'count': total_count.values})

# 빈도수가 100 이상인 단어만 추출
df_result = df_total_count[df_total_count['count'] >= 100]

# # 결과 출력
# print(df_result)

In [ ]:
df_result["count"] = df_result["count"].astype(int) # count컬럼에 데이터가 flost타입이라서 int타입으로 변경
df_result["count"].sum() # 총 단어갯수 확인
df_result[["count"]] = df_result[["count"]].apply(lambda x : x / 198563) # 단어총갯수와 한단어씩 갯수 나누기 

In [ ]:
df = pd.read_excel("/content/전체언론사 단어 빈도수 평균.xlsx")

df['two_word'] = df['word'].apply(lambda x: x[:3] if len(x) > 1 else None) # two_word 컬럼을만들어서 3단어이상 전처리

df = df.dropna(subset=['two_word']) # two_word 컬럼에서 Nan값 제거

In [ ]:
df = df.drop(df[df['two_word'] == 'vs'].index) # two_word 컬럼에잇는 데이터중 vs결측치 제거

# 부동산 심리지수 분기별로 평균 전처리


In [ ]:
import pandas as pd
df = pd.read_excel("/content/drive/MyDrive/1차프로젝트 데이터 자료/부동산시장_소비심리지수_부동산원 (1).xlsx") # 엑셀파일 불러오기
df

In [ ]:
df.drop(columns=['전국'],inplace=True) # 필요없는 컬럼 제거

In [ ]:
df.rename(columns={'수도권': 'mean'}, inplace=True)# 컬럼명을 수도권에서 mean로 컬럼명 변경 
df.rename(columns={'시점': 'date'}, inplace=True) # 컬럼명을 시점에서 date로 컬럼명 변경 

In [ ]:
df = df.drop(df[df['date'] == '시점'].index) # dete 컬럼에잇는 시점 인덱스 제거
df = df.drop(df[df['mean'] == '서울특별시'].index) # mean 컬럼에잇는 서울특별시 인덱스 제거

In [ ]:
df['date'] = pd.to_datetime(df['date'])

df_grouped = df.groupby(pd.PeriodIndex(df['date'], freq='Q')).mean() # date 에잇는 시계열데이터 각분기(Q)순으로 데이터수정
df_grouped 

In [ ]:
df_people['date'] = pd.to_datetime(df_people['date'], format='%Y-%m-%d') # people를 데이터타임 형태로 바꾸고 
df_people['date'] = df_people['date'].dt.strftime('%Y-%m')# 연도와 월만 추출되어 새로운 문자열로 변경

In [ ]:
df_people['date'][0][-2:]# df_people의 'date' 컬럼에서 첫 번째 데이터의 뒤에서 두 자리 숫자를 추출

In [ ]:
change = []
for i in range(len(df_people['date'])):
  if df_people['date'][i][-2:] == '01':
    change.append(df_people['date'][i][:4] + ' 1분기')
  elif df_people['date'][i][-2:] == '04':
    change.append(df_people['date'][i][:4] + ' 2분기')
  elif df_people['date'][i][-2:] == '07':
    change.append(df_people['date'][i][:4] + ' 3분기')
  else:
    change.append(df_people['date'][i][:4] + ' 4분기')

# 위 코드는 df_people 데이터프레임의 'date' 컬럼에서 월 정보를 추출하여 해당 월이 속하는 분기 정보로 변환하는 과정.
# '2021-01'은 '2021 1분기'로 변환됩니다. 변환된 분기 정보는 change 리스트에 저장됩니다.

In [ ]:
df_people['date'] = change # 날짜 데이터의 마지막 두 글자를 기반으로 1분기부터 4분기까지의 분기 정보를 추가한 후 데이터프레임에 적용
df_people

In [ ]:
df_new = pd.read_excel("/content/월간 아파트 지수 통계.xlsx")
df_new

In [ ]:
df_avg = df_new.groupby(np.arange(len(df_new)) // 3).apply(lambda x: x.mean()) # df_new의 각 row를 3개씩 묶어서 그룹을 만들고, 각 그룹마다 평균값을 구해 새로운 데이터프레임 df_avg를 만들어줍니다.
df_avg

In [ ]:
df_concat = pd.concat([df_people, df_avg['서울']], axis = 1) # df_people,df_avg["서울"]을 열뱡향으로 합침
df_concat

## 웹크롤링 본문(content) 상승과 단어 빈도수 전처리 시각화


# 2018년~2019년 상승단어/하락단어 빈도수 정리

In [ ]:
import pandas as pd

# 데이터프레임 불러오기
df = pd.read_excel('/content/drive.xlsx')

# content 열을 문자열로 변환하여 하나의 문자열로 합침
content_str = ' '.join(df['content'].astype(str).tolist())

# 2018년 1분기, 2분기, 3분기, 4분기로 분할
quarters = ['2018년 1분기', '2018년 2분기', '2018년 3분기', '2018년 4분기']
quarter_contents = [content_str[:len(content_str)//4],
                    content_str[len(content_str)//4:len(content_str)//2],
                    content_str[len(content_str)//2:3*len(content_str)//4],
                    content_str[3*len(content_str)//4:]]

# 분기별 상승/하락 단어 빈도수 계산
up_counts = []
down_counts = []
for content in quarter_contents:
    up_count = content.count('상승')
    down_count = content.count('하락')
    up_counts.append(up_count)
    down_counts.append(down_count)

# 결과 데이터프레임 생성
result_df = pd.DataFrame({'분기': quarters, '상승 단어 빈도수': up_counts, '하락 단어 빈도수': down_counts})
result_df_1 = pd.DataFrame({'분기': [quarters[0]], '상승 단어 빈도수': [up_counts[0]], '하락 단어 빈도수': [down_counts[0]]})
result_df_2 = pd.DataFrame({'분기': [quarters[1]], '상승 단어 빈도수': [up_counts[1]], '하락 단어 빈도수': [down_counts[1]]})
result_df_3 = pd.DataFrame({'분기': [quarters[2]], '상승 단어 빈도수': [up_counts[2]], '하락 단어 빈도수': [down_counts[2]]})
result_df_4 = pd.DataFrame({'분기': [quarters[3]], '상승 단어 빈도수': [up_counts[3]], '하락 단어 빈도수': [down_counts[3]]})

# 2020년~2022년 상승단어/하락단어 빈도수 정리

In [ ]:
import pandas as pd

# 데이터프레임 불러오기
df = pd.read_excel('/content/drive1.xlsx')

# content 열을 문자열로 변환하여 하나의 문자열로 합침
content_str = ' '.join(df['content'].astype(str).tolist())

# 2020년 1분기, 2분기, 3분기, 4분기로 분할
quarters = ['2020년 1분기', '2020년 2분기', '2020년 3분기', '2020년 4분기',
            '2021년 1분기', '2021년 2분기', '2021년 3분기', '2021년 4분기',
            '2022년 1분기', '2022년 2분기', '2022년 3분기', '2022년 4분기']
quarter_contents = [content_str[:len(content_str)//12],
                    content_str[len(content_str)//12:len(content_str)//6],
                    content_str[len(content_str)//6:3*len(content_str)//12],
                    content_str[3*len(content_str)//12: len(content_str)//3],
                    content_str[len(content_str)//3: 5*len(content_str)//12],
                    content_str[5*len(content_str)//12: len(content_str)//2],
                    content_str[len(content_str)//2:7*len(content_str)//12],
                    content_str[7*len(content_str)//12: 4*len(content_str)//6],
                    content_str[4*len(content_str)//6: 9*len(content_str)//12],
                    content_str[9*len(content_str)//12: 5*len(content_str)//6],
                    content_str[5*len(content_str)//6: 11*len(content_str)//12],
                    content_str[11*len(content_str)//12:]]

# 분기별 상승/하락 단어 빈도수 계산
up_counts = []
down_counts = []
for content in quarter_contents:
    up_count = content.count('상승')
    down_count = content.count('하락')
    up_counts.append(up_count)
    down_counts.append(down_count)

# 결과 데이터프레임 생성
result_df_2 = pd.DataFrame({'분기': quarters, '상승 단어 빈도수': up_counts, '하락 단어 빈도수': down_counts})

In [ ]:
result_new = pd.concat([result_df, result_df_2], axis=0, ignore_index=True) # result_df와 result_df_2를 위아래로 합쳐서  ignore_index=True 사용을해서 인덱스가 순서대로 되도록 진행

# 2018년~2022년 상승단어/하락단어 부동산심리 수치  선그래프 시각화 


In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 7))

# y1, y2에 대한 라인 플롯 그리기
ax1.plot(x, y1, label=y1.name, color='red')
ax1.plot(x, y2, label=y2.name, color='blue')

# ax1과 독립적인 y축을 가지는 ax2 설정
ax2 = ax1.twinx()
# 부동산심리지수(x_mean, y_mean)에 대한 라인 플롯 그리기
ax2.plot(x_mean, y_mean, label='부동산심리지수', color='black')

# 축 라벨 설정
ax1.set_xlabel('분기')
ax1.set_ylabel('단어 빈도수', position=(0, 0.45))  # y축 라벨의 위치 조정
ax2.set_ylabel('부동산심리지수', position=(1, 0.45))  # y축 라벨의 위치 조정

# x축 설정
ax1.set_xticks(x)
ax1.set_xticklabels(df["분기"], rotation=90)  # x축 라벨을 90도 회전하여 출력

# 범례 설정
fig.legend(bbox_to_anchor=(0.5, -0.1), loc='lower center', ncol=3)

# 마커 추가
ax1.plot(x, y1, label=y1.name, color='red', marker='o', markersize=10)
ax1.plot(x, y2, label=y2.name, color='blue', marker='o', markersize=10)
ax2.plot(x_mean, y_mean, label='부동산심리지수', color='black', marker='o', markersize=10)

# 타이틀 설정
plt.title('2018년~2022년 상승단어/하락단어 부동산심리 수치')

# 그래프 출력
plt.show()


# 2018년~2022년 상승단어/하락단어 부동산심리 수치 바그래프 시각화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 그래프 크기를 지정하여 figure와 axis 객체 생성
fig, ax = plt.subplots(figsize=(23, 7))

# 막대 그래프의 너비를 설정
bar_width = 0.35

# 막대 그래프 x축 위치를 위한 배열 생성
r1 = np.arange(len(result_new))

# result_new DataFrame의 "상승 단어 빈도수" 열을 이용해 막대 그래프 생성 (빨간색)
ax.bar(r1, result_new['상승 단어 빈도수'], label='상승', color='red', width=bar_width)

# 왼쪽 y축 레이블 설정
ax.set_ylabel('상승 단어 빈도수')

# result_new DataFrame의 "하락 단어 빈도수" 열을 이용해 막대 그래프 생성 (파란색)
ax.bar(r1 + bar_width, result_new['하락 단어 빈도수'], label='하락', color='blue', width=bar_width)

# 왼쪽 y축 레이블 설정
ax.set_ylabel('하락 단어 빈도수')

# 새로운 y축을 만들어서 부동산심리지수를 라인 그래프로 추가
ax2 = ax.twinx()
ax2.plot(df_concat['mean'], label='부동산심리지수', color='black')

# x축 눈금 위치 지정하고 눈금 레이블을 result_new DataFrame의 "분기" 열 값으로 설정 (90도 회전)
ax.set_xticks(r1 + bar_width / 2)
ax.set_xticklabels(result_new['분기'], rotation=90)

# 왼쪽 y축과 오른쪽 y축의 범례를 하나로 합쳐서 위치를 "upper left"로 지정
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

# 그래프 제목 설정
plt.title('2018년~2022년 상승단어/하락단어 부동산심리 수치')

# 그래프 출력
plt.show()

# 월간 아파트 지수 통계 삼전도 추세선 그래프 시각화

In [ ]:
df_concat['서울 심리'] = df_concat['서울 심리'].astype(float) # df_concat["서울 심리"] 컬럼을 float타입으로 변경
df_concat.info()

# x축기준으로 

In [ ]:
# 그래프 그리기
sns.regplot(data=df_concat, x = '서울', y = '서울 심리')

# x축, y축 설정
plt.xlabel('서울 월간 아파트 지수')
plt.ylabel('서울 심리 지수')

# 그래프 제목
plt.title('2018 ~ 2022 서울 월별 아파트 지수와 부동산심리지수')

# 그래프 보여 주기
plt.show()

# y축 기준으로

In [ ]:
# 그래프 그리기
sns.regplot(data=df_concat, x = '서울 심리', y = '서울')

# x축, y축 설정
plt.xlabel('서울 심리 지수')
plt.ylabel('서울 월간 아파트 지수')

# 그래프 제목
plt.title('2018 ~ 2022 서울 월별  부동산심리지수와 아파트 지수')

# 그래프 보여 주기
plt.show()

# 추가로 타이틀(title)기준으로 단어 전처리및 시각화

In [ ]:
import pandas as pd
from konlpy.tag import Okt

# okt 객체 생성
okt = Okt()

# content 열에서 Null 값 제거
df = df.dropna(subset=['title'])

# content 열의 모든 문장에 대해 형태소 분석을 수행하고, 그 결과를 'morphs' 열에 저장
df['morphs'] = df['title'].apply(lambda x: okt.morphs(str(x)))

# 단어 집합(vocabulary) 생성
vocabulary = {}
for i in df['morphs']:
    for j in i:
        if j not in vocabulary:
            vocabulary[j] = 0
        vocabulary[j] += 1

# 빈도수가 높은 순으로 정렬된 단어 리스트 생성
word_list = sorted(vocabulary.items(), key=lambda x: x[1], reverse=True)

# 결과를 엑셀 파일로 저장(단어 포함)
result_df = pd.DataFrame(word_list, columns=['word', 'freq'])
result_df.to_excel('./2019년(단어 포함) 본문 단어 빈도수.xlsx', index=False)


# 결과를 엑셀 파일로 저장(단어 미포함)
word_list = sorted([(k, v) for k, v in vocabulary.items() if k not in ['서울', '아파트', '매매']], key=lambda x: x[1], reverse=True) # '서울', '아파트', '매매'를 제외한 vocabulary 딕셔너리에서 빈도수가 높은 순으로 정렬된 단어 리스트 생성
result_df = pd.DataFrame(word_list, columns=['word', 'freq'])
result_df.to_excel('./2019년(단어 미포함) 본문 단어 빈도수.xlsx', index=False)

In [ ]:
df = result_df

In [ ]:
# 만약에 freq 타입이 int가 아닐 경우
df['freq'] = df['freq'].astype(int)

In [ ]:
# 같은 단어의 freq를 합침
df = pd.DataFrame(df.groupby(['word'])['freq'].sum()).reset_index()

# 웹크롤링 워드클라우드 시각화 

In [ ]:
# 시각화(워드클라우드)
# 본문 단어 빈도수(단어 포함)
!pip install wordcloud

import pandas as pd
from konlpy.tag import Okt
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.rc('font', family = ' NanumGothic')

# 엑셀 파일 불러오기
df = pd.read_excel('/content/2019.02.xlsx')

# '본문' 컬럼의 텍스트 데이터 추출
text_data = ''
for text in df['word']:
    text_data += str(text) + ' '

# 형태소 분석기를 사용하여 명사 추출
okt = Okt()
nouns = okt.nouns(text_data)

# 단어 빈도수 계산
freq_dict = {}
for noun in nouns:
    if len(noun) > 1:
        freq_dict[noun] = freq_dict.get(noun, 0) + 1

# 단어 빈도수 데이터프레임으로 변환
df_freq = pd.DataFrame(list(freq_dict.items()), columns=['word', 'freq'])

# 한글 폰트 경로 지정
font_path = '/content/NanumGothic.ttf'

# 워드 클라우드 생성
wordcloud = WordCloud(font_path=font_path, background_color='white',collocations=False, regexp="[\uAC00-\uD7A3]+").generate_from_frequencies(dict(zip(df_freq['word'], df_freq['freq'])))

# 워드 클라우드 출력
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show() 